In [1]:
# 파이썬 시각화 패키지 불러오기
import matplotlib.pyplot as plt
%matplotlib inline
# 사용자 운영체제 확인
import platform
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [2]:
import re
import numpy as np
import pandas as pd
import json
import requests
import branca
import folium
from folium import plugins
folium.__version__

'0.11.0'

In [3]:
df=pd.read_csv("jeju_list.csv")

In [5]:
df.fillna('-',inplace=True)

In [6]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   가게명         256 non-null    object 
 1   대분류         256 non-null    object 
 2   중분류         256 non-null    object 
 3   소분류         256 non-null    object 
 4   mean_score  256 non-null    float64
 5   주소          256 non-null    object 
 6   Latitude    256 non-null    float64
 7   Longitude   256 non-null    float64
dtypes: float64(3), object(5)
memory usage: 16.1+ KB


,가게명,대분류,중분류,소분류,mean_score,주소,Latitude,Longitude
0,상춘재,식당,한식,"돌문어 비빔밥, 게장",4.1,제주특별자치도 제주시 조천읍 선진길 26,33.458659,126.704844
1,남양수산,식당,일식,"회, 고등어",4.6,제주특별자치도 서귀포시 성산읍 고성동서로56번길 11,33.450135,126.914224
2,선흘곶,식당,한식,정통 한정식,4.5,제주특별자치도 제주시 조천읍 동백로 102,33.508187,126.718049
3,스시호시카이,식당,일식,"회, 초밥, 오마카세",4.6,제주특별자치도 제주시 오남로 90,33.481286,126.516769
4,미영이네식당,식당,일식,"회, 고등어, 방어, 참치",4.5,제주특별자치도 서귀포시 대정읍 하모항구로 42,33.217651,126.249904


In [7]:
jeju_df=pd.read_excel("jejuSIG.xlsx")
jeju_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  467 non-null    int64  
 1   시군구         467 non-null    object 
 2   전용/연면적      467 non-null    float64
 3   거래금액        467 non-null    int64  
 4   평당 거래금액     467 non-null    int64  
 5   코드          467 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 22.0+ KB


In [8]:
jeju_df.drop(['Unnamed: 0'], axis=1, inplace=True)

# 제주 "읍면동" 평균 거래금액

jeju_EMD=jeju_df[jeju_df['코드']%100==0]
jeju_EMD

##### 이렇게 하면 "리" 가 포함된 "읍면동"도 제거되서 "리"만 따로 자세히 뽑을때 이 기법을 사용함
##### 리를 포함한 읍면동을 평균으로 뽑으려면 다른방법 사용

In [65]:
jeju_df['코드']=jeju_df['코드']//100

In [66]:
EMD_mean_price=jeju_df.groupby(['코드'])['거래금액'].mean().round(0)

코드
50110101    136302.0
50110102     32269.0
50110103     59242.0
50110104     71972.0
50110105     25025.0
50110106     30978.0
50110107     29465.0
50110108     27860.0
50110109     18000.0
50110110     34333.0
50110111     28760.0
50110112     75909.0
50110113     27750.0
50110114     35958.0
50110117     90000.0
50110118     13333.0
50110119     39000.0
50110120     46500.0
50110122    132875.0
50110123    174881.0
50110126     49900.0
50110127     40000.0
50110129     17964.0
50110130     29878.0
50110131     35000.0
50110133      4963.0
50110134    492966.0
50110136     56000.0
50110137     89191.0
50110138    115300.0
50110139    362750.0
50110250     75686.0
50110253     71880.0
50110256     26667.0
50110259     37865.0
50110310     50717.0
50110320      8750.0
50130101     26449.0
50130102     40707.0
50130103     33352.0
50130104     28000.0
50130105     28910.0
50130106     31500.0
50130108     50000.0
50130111    185100.0
50130112    127705.0
50130116    122000.0
50130250  

In [67]:
EMD=pd.DataFrame(EMD_mean_price)

In [68]:
EMD.reset_index(inplace=True)

In [69]:
EMD['코드']=EMD['코드'].astype(str)

In [71]:
EMD

,코드,거래금액
0,50110101,136302.0
1,50110102,32269.0
2,50110103,59242.0
3,50110104,71972.0
4,50110105,25025.0
5,50110106,30978.0
6,50110107,29465.0
7,50110108,27860.0
8,50110109,18000.0
9,50110110,34333.0


In [72]:
EMD.to_csv("EMD.csv", index=False, encoding='utf-8-sig')

# 제주 "리" 평균 거래금액

In [18]:
jeju_LI=jeju_df[jeju_df['코드']%100!=0]
jeju_LI

,시군구,전용/연면적,거래금액,평당 거래금액,코드
2,제주특별자치도 서귀포시 남원읍 위미리,395.01,210000,531,5013025327
3,제주특별자치도 서귀포시 남원읍 위미리,40.80,13500,330,5013025327
4,제주특별자치도 서귀포시 남원읍 위미리,40.80,15500,379,5013025327
5,제주특별자치도 서귀포시 남원읍 위미리,62.40,13000,208,5013025327
6,제주특별자치도 서귀포시 남원읍 위미리,62.40,14200,227,5013025327
...,...,...,...,...,...
428,제주특별자치도 제주시 한림읍 대림리,2943.87,400000,135,5011025023
429,제주특별자치도 제주시 한림읍 옹포리,110.33,27000,244,5011025028
430,제주특별자치도 제주시 한림읍 옹포리,15.91,5500,345,5011025028
431,제주특별자치도 제주시 한림읍 월림리,72.44,28000,386,5011025034


In [23]:
LI_mean_price=jeju_LI.groupby(['코드'])['거래금액'].mean().round(0)
LI=pd.DataFrame(LI_mean_price)

In [45]:
LI.reset_index(inplace=True)

In [60]:
#읍면동은 geojson에서 코드 맨뒤 00 생략 그리고 코드는 str타입임..
# for emd in EMD['코드']:
LI['코드']=LI['코드'].astype(str)

In [61]:
LI['코드']

0     5011025023
1     5011025024
2     5011025025
3     5011025028
4     5011025033
5     5011025034
6     5011025321
7     5011025326
8     5011025331
9     5011025333
10    5011025334
11    5011025338
12    5011025340
13    5011025341
14    5011025628
15    5011025630
16    5011025631
17    5011025633
18    5011025634
19    5011025921
20    5011025923
21    5011025924
22    5011025926
23    5011025930
24    5011031021
25    5011031022
26    5011031023
27    5011031024
28    5011031027
29    5011032021
30    5011032025
31    5013025021
32    5013025022
33    5013025023
34    5013025025
35    5013025029
36    5013025327
37    5013025926
38    5013025927
39    5013025928
40    5013031025
41    5013031027
42    5013031028
43    5013032021
44    5013032022
45    5013032023
46    5013032024
47    5013032026
Name: 코드, dtype: object

In [62]:
LI.to_csv("LI.csv", index=False, encoding='utf-8-sig')